# Imports

In [1]:
# # ! pip install pandas-profiling
# ! pip install geopandas
# # ! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
# ! pip install pandas
# # ! pip install pandas==0.25
# ! pip install tensorflow_text
# # !wget http://nlp.stanford.edu/data/glove.6B.zip
# # !unzip -q glove.6B.zip

In [2]:
# 'imports; megabytes of imports'  - Johnny 5
import pandas as pd
import numpy as np
# from google.colab import files
import pandas_profiling
import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium.plugins import FastMarkerCluster, Fullscreen, MiniMap, HeatMap, HeatMapWithTime
import geopandas as gpd
import os
import re
import shutil
import string
import operator
import pathlib


# import tensorflow as tf
# import tensorflow_datasets as tfds
# import tensorflow_hub as hub
# from tensorflow.keras import layers
# from tensorflow.keras import losses
# from tensorflow.keras import preprocessing
# from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
# import tensorflow_text as tf_text
# from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

from sklearn.metrics import log_loss, make_scorer, confusion_matrix, ConfusionMatrixDisplay
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.model_selection import train_test_split,cross_val_score,  GridSearchCV
from sklearn.feature_selection import f_classif, chi2
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import StandardScaler

# from google.colab import drive
# drive.mount('/content/drive')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/scottsturtz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
pd.set_option('display.max_colwidth', None) 

In [4]:
# import dataset
tdata = pd.read_csv('globalterrorismdb_0718dist.csv', low_memory=False)
# tdata = pd.read_csv('/content/drive/MyDrive/globalterrorismdb_0718dist.csv', engine='python')
# tdata.set_index('eventid')
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [5]:
# pandas_profiling.ProfileReport(tdata)

# Data EDA and Cleaning

In [6]:
# columns that can be removed: 
out_columns =  ['weapsubtype1', 'weaptype1', 'claimed',   'nperpcap', 'nwoundus', 'propvalue', 'propcomment', 'addnotes', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'related',  'property',  'propextent','nperps', 'natlty1', 'targsubtype1','motive','dbsource','approxdate' , 'extended' , 'resolution' , 'vicinity' , 'location' , 'alternative', 'alternative_txt', 'multiple', 'attacktype2', 'attacktype2_txt' , 'attacktype3' ,'attacktype3_txt' ,'targtype2' ,	'targtype2_txt' , 	'targsubtype2'	, 'targsubtype2_txt' ,	'corp2' , 	 'target2',	'natlty2',	'natlty2_txt','targtype3',	'targtype3_txt',	'targsubtype3',	'targsubtype3_txt',	'corp3',	'target3',	'natlty3',	'natlty3_txt',	'gsubname',	'gname2',	'gsubname2',	'gname3',	'gsubname3',	'guncertain2',	'guncertain3', 'claimmode',	'claimmode_txt',	'claim2',	'claimmode2',	'claimmode2_txt','claim3',	'claimmode3',	'claimmode3_txt',	'compclaim', 'weaptype2',	'weaptype2_txt',	'weapsubtype2',	'weapsubtype2_txt',	'weaptype3', 'weapsubtype3',	'weapsubtype3_txt', 'weaptype4','weaptype4_txt',	'weapsubtype4',	'weapsubtype4_txt', 'nkillus',	'nkillter',	'nwound',	'nwoundte', 'ishostkid',	'nhostkid',	'nhostkidus',	'nhours',	'ndays',	'divert', 'kidhijcountry',	'ransom',	'ransomamt',	'ransomamtus',	'ransompaid',	'ransompaidus',	'ransomnote',	'hostkidoutcome',	'hostkidoutcome_txt', 'nreleased', 'weaptype3_txt' ]

tdata = tdata.drop(columns = out_columns) 
tdata.fillna('', inplace=True)

In [7]:

counts = tdata['gname'].value_counts()
tdata = tdata[tdata['gname'].isin(counts[counts > 400].index)]
tdata.drop(tdata[tdata['summary'] == ''].index, inplace = True)
tdata['summary']= tdata['summary'].str.replace('\d+', '') #remove digits
tdata['summary']= tdata['summary'].str.replace('[^\w\s]', '')#remove punctuation

<ipython-input-7-071a8e781505>:4: FutureWarning: The default value of regex will change from True to False in a future version.
  tdata['summary']= tdata['summary'].str.replace('\d+', '') #remove digits
<ipython-input-7-071a8e781505>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  tdata['summary']= tdata['summary'].str.replace('[^\w\s]', '')#remove punctuation


In [8]:
#copies to work with
df= tdata.copy() # used during assignment
df2=tdata.copy() # further experimenting
df2=tdata.select_dtypes(exclude=['int64'])


In [9]:
df2= df2.replace('\d+', '') #remove digits
df2= df2.replace('[^\w\s]', '')#remove punctuation

# df.isnull().sum()
# df2.info()

# df['data'] = df[['country_txt', 'region_txt ','provstate', 'city', '', '']].agg('-'.join, axis=1)

In [10]:
# data to predict on after training
need_predict = df[df['gname'] == 'Unknown']
# need_predict.info
# need_predict.head()

In [11]:
#remove prediction data from main dataframe
df.drop(df[df['gname'] == 'Unknown'].index, inplace = True)
df2.drop(df2[df2['gname'] == 'Unknown'].index, inplace = True)

In [12]:
# reported groups
# len(df.gname.value_counts())

In [13]:
#create 10% holdback tp verify multiple models. 
#Don't touch until end!
hold_back = df.sample(frac = 0.1)
hold_back2 = df2.sample(frac = 0.1)
#hold_back.shape

In [14]:
df = df.drop(hold_back.index)
df2 = df2.drop(hold_back2.index)

In [15]:
# df.shape,df2.shape


# Visuals

Top 30 active Terrorist Groups

In [16]:
# Groups = df['gname'].value_counts()
# # rank = Groups[1].rank(ascending=False).values
# # rank = (rank-1).astype(np.int)
# GroupList = sorted(list(zip(Groups.values[:35],Groups.index[:35])),reverse=True)
# MostActive, Counts = zip(*GroupList)
# MostActive, Counts = list(MostActive), list(Counts)

# fig, ax = plt.subplots(figsize=(20,15))
# t = sns.barplot(x=MostActive, y=Counts, palette='flare' )
# # sns.barplot(x=Counts,y=MostActive)
# ax.set_title('Most Active Terrorist Groups', fontsize=20)
# plt.xlabel("Attacks", fontsize=16)
# plt.rcParams['font.family'] = 'DejaVu Sans'
# plt.yticks(fontsize=16)
# plt.xticks(np.arange(0, 7001, 200), rotation=60, fontsize=13)
# fig.patch.set_facecolor('lightgray')
# ax.set_facecolor('azure')
# plt.tight_layout()
# for v in str(Groups['count']):
# plt.savefig('Most_Active.png')
# plt.show()
# Groups['count']


Total Attacks vs Year



In [17]:
# tdataYear = list(zip(tdata.iyear.value_counts().index,tdata.iyear.value_counts().values))
# tdataYear = sorted(tdataYear)
# terrorByYear = []
# terrorByCounts = []
# for i in tdataYear:
#     terrorByYear.append(i[0])
#     terrorByCounts.append(i[1])

# dfYear = list(zip(df.iyear.value_counts().index,df.iyear.value_counts().values))
# dfYear = sorted(dfYear)
# terrorByYear2 = []
# terrorByCounts2 = []
# for i in dfYear:
#     terrorByYear.append(i[0])
#     terrorByCounts.append(i[1])

# fig, ax = plt.subplots(figsize=(20,12))
# sns.pointplot(x=terrorByYear,y=terrorByCounts,palette=sns.color_palette("icefire_r"))
# ax.set_title('Activity/Year Full Data', fontsize=20)
# plt.xticks(fontsize=15, rotation=70)
# plt.xlabel("Year",fontsize=18 )
# plt.ylabel("# of Attacks",fontsize=18)
# plt.rcParams['font.family'] = 'DejaVu Sans'
# plt.yticks(np.arange(0.5,15001, 500),fontsize=15)
# fig.patch.set_facecolor('lightgray')
# ax.set_facecolor('azure')
# plt.grid()



# plt.tight_layout()
# plt.savefig('Attack Escalation Full.png')
# # plt.show()


Attacks By Year and Region

In [18]:

# regionYear = pd.crosstab(df.iyear,df.region_txt)
# regionYear.plot(color=sns.color_palette("icefire_r"),grid=True)

# fig=plt.gcf()
# fig.set_size_inches(20,8)
# plt.xlabel("Year",fontsize=18 )
# plt.ylabel("Attacks",fontsize=18)
# plt.xticks(np.arange(1970, 2017.1, 2),fontsize=15, rotation=70)
# plt.title("Attacks by Year and Region", fontsize=20)
# plt.rcParams['font.family'] = 'DejaVu Sans'
# plt.yticks(np.arange(0.5,2501, 200),fontsize=15)
# fig.patch.set_facecolor('lightgray')
# ax.set_facecolor('azure')
# plt.tight_layout()
# plt.show()


# plt.grid()
# plt.savefig('Region_Attacks.png') 

Heat Map

In [19]:
# country = tdata['country_txt']
# country_weapon = tdata.groupby(by='weaptype1_txt', as_index=False).count().sort_values(by='eventid',ascending=False).iloc[:,:2]
                                                  
                                                                                                                
# sns.barplot(x='weaptype1_txt', y='eventid', data=country_weapon, ax=ax, ci=None)
# ax[1, 1].set_xticklabels(ax[1, 1].get_xticklabels(), rotation=90)
# ax[1, 1].set_xlabel('')
# ax[1, 1].set_ylabel('Count')
# format_spines(ax[1, 1], right_border=False)
# ax[1, 1].set_title(f'{country_name} Weapons Used in Attacks')

# plt.suptitle(f'Terrorism Analysis in {country_name} between 1970 and 2017', size=16)
# plt.tight_layout()
# plt.subplots_adjust(top=0.90)
# plt.show()

In [20]:
# pandas_profiling.ProfileReport(df)

In [21]:
counts2 = df['gname'].value_counts()
# print(counts2)
# counts.head(40)

In [22]:
# df2.head(100)

# Models

In [23]:
#create train test split from df
y = df['gname']
X = df.drop(columns= ['eventid','longitude', 'latitude', 'gname'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

In [24]:
df2['name_id'] = df2['gname'].factorize()[0]

g_data = df2[['gname', 'name_id']].drop_duplicates().sort_values('name_id')

gname2id= dict(g_data.values)
id2gname= dict(g_data[['name_id', 'gname']]).values
y2= df2['gname']

X2 = df2.drop(columns= ['longitude', 'latitude', 'gname', 'specificity', 'doubtterr', 'guncertain1', 'nkill','scite3'])
X2['all_data'] = X2[['country_txt','region_txt','provstate','city','summary','attacktype1_txt','targtype1_txt','targsubtype1_txt','corp1','target1','natlty1_txt','weaptype1_txt','weapsubtype1_txt','weapdetail','propextent_txt','scite1','scite2']].agg('- '.join, axis=1)
X2['all_data']= X2['all_data'].str.replace('\d+', '') #remove digits
X2['all_data']= X2['all_data'].str.replace('[^\w\s]', '')#remove punctuation
X2['all_data'].str.strip()# remove extra whitespace
X2_train, X2_test, y2_train, y2_test = train_test_split(X2['all_data'],y2, test_size=.30, random_state=42) 
# list(X2.columns.values)
# X2.head(2)

<ipython-input-24-54b71cfcad98>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  X2['all_data']= X2['all_data'].str.replace('\d+', '') #remove digits
<ipython-input-24-54b71cfcad98>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  X2['all_data']= X2['all_data'].str.replace('[^\w\s]', '')#remove punctuation


## Random Forest 

In [25]:
rfd = X_train['summary']
pred_rfd = X_test['summary']
y_train = y_train
y_test = y_test
#vectorize summary data
new_stopwords=['faraj',
'farc' ,'farah', 'shewan','zuweid']
stpwrd = stopwords.words('english')
stpwrd.extend(new_stopwords)

vectorizer = CountVectorizer(lowercase=True, stop_words='english', max_features = 100)
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words=stpwrd )


vectorizer.fit(rfd)
blind = need_predict.head()


# RFX_train = vectorizer.transform(rfd)
# RFX_test = vectorizer.transform(pred_rfd)
# RFX_blind = vectorizer.transform(blind)
# X2_train= vectorizer.transform(X2_train)
# X2_test = vectorizer.transform(X2_test)

features = tfidf.fit_transform(X2_train).toarray()
labels = y2_train
features

array([[0.11621966, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [26]:
# RFX_train.shape
X2_train.shape

(23041,)

In [27]:
#need to troubleshoot. Why am I getting same words for all
N=2
for gname, name_id in sorted(gname2id.items()):
  features_chi2 = chi2(features, labels == name_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(gname))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:]))) 

# 'Abu Sayyaf Group (ASG)':
  . Most correlated unigrams:
. fiftyeight
. fight
  . Most correlated bigrams:
. fighters seize
. zurmat district
# 'Al-Qaida in Iraq':
  . Most correlated unigrams:
. fiftyeight
. fight
  . Most correlated bigrams:
. fighters seize
. zurmat district
# 'Al-Qaida in the Arabian Peninsula (AQAP)':
  . Most correlated unigrams:
. fiftyeight
. fight
  . Most correlated bigrams:
. fighters seize
. zurmat district
# 'Al-Shabaab':
  . Most correlated unigrams:
. fiftyeight
. fight
  . Most correlated bigrams:
. fighters seize
. zurmat district
# 'Basque Fatherland and Freedom (ETA)':
  . Most correlated unigrams:
. fiftyeight
. fight
  . Most correlated bigrams:
. fighters seize
. zurmat district
# 'Boko Haram':
  . Most correlated unigrams:
. fiftyeight
. fight
  . Most correlated bigrams:
. fighters seize
. zurmat district
# 'Communist Party of India - Maoist (CPI-Maoist)':
  . Most correlated unigrams:
. fiftyeight
. fight
  . Most correlated bigrams:
. fighter

In [28]:
# Train
RFC = RandomForestClassifier(n_estimators=40, max_depth=20)
RF = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',max_depth=None, max_features='auto', max_leaf_nodes=None,min_impurity_decrease=0.0, min_impurity_split=None,min_samples_leaf=1, min_samples_split=2,min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,oob_score=False, random_state=None, verbose=0,warm_start=False)



# RFC.fit(RFX_train, y_train)
# RFC.fit(X2_train, y2_train)
RF.fit(X2_train, y2_train)

ValueError: could not convert string to float: 'Yemen Middle East  North Africa Marib Marib  Assailants launched a missile targeting AlRuweik military camp in Marib Yemen There were no reported casualties in the attack Houthi extremists claimed responsibility for the incident BombingExplosion Military Military BarracksBaseHeadquartersCheckpost Military of Yemen Ar Ruwayk Camp Yemen Explosives Projectile rockets mortars RPGs etc A ballistic missile was used in the attack  Saudiled coalition targets Houthi launchpads in Yemen Anadolu Agency AA December   Yemeni forces missile hit Saudi base in Marib prov GlobalSecurityorg December  '

In [27]:
y_pred = RF.predict(X2_test)
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y2_test, y_pred))

Accuracy: 0.9540253164556962


Accuracy: 0.9808607594936709


In [28]:
pred = RFC.predict(RFX_blind)

# print(pred)

In [29]:
# #need_predict
# blindtest = need_predict.head(5)
# blindtest=vectorizer.transform(blindtest['summary'])
# test2 = RFC.predict(blindtest)

In [30]:
# feature importance
feature_imp = X_train(RFC.feature_importances_,index=X_train(ascending=False))

TypeError: 'DataFrame' object is not callable

In [30]:
# result = permutation_importance(RFC, RFX_train.toarray(), y_train, n_repeats=2,random_state=42, n_jobs=2)
# sorted_idx = result.importances_mean.argsort()

# fig, ax = plt.subplots()
# ax.boxplot(result.importances[sorted_idx].T,
#            vert=False, labels=RFX_train.columns[sorted_idx])
# ax.set_title("Permutation Importances (train set)")
# fig.tight_layout()
# plt.show()
# result = permutation_importance(RFC, RFX_test.toarray(), y_test)
# def permutation_importance(X, y, RFC): 
#     perm = {}
#     y_true = model.predict_proba(X)[:,1]
#     baseline= roc_auc_score(y, y_true)
#     for cols in X.columns:
#         value = X[cols]
#         X[cols] = np.random.permutation(X[cols].values)
#         y_true = model.predict_proba(X)[:,1]
#         perm[cols] = roc_auc_score(y, y_true) - baseline
#         X[cols] = value
#     return perm
# prmutation_importance(RFX_test.toarray(), y_test, RFC)
# parameters = {'max_depth' : np.arange(1, 12),
#              'min_samples_leaf' : np.arange(5,95,10)}
             
# scorer = make_scorer(log_loss,
#                      greater_is_better=False,
#                      needs_proba=True)
# clf = GridSearchCV(RFC,
#                    parameters,
#                    cv=10,
#                    scoring=scorer).fit(RFX_train,y_train)
# #Train and fit model                                                   
# rf = RandomForestClassifier(n_estimators=1000,
#                            max_features='auto',
#                            random_state=0)
# rf.fit(RFX_train, y_train)
                                     
# Test Prediction
# pred = rf.predict(RFX_test)



# cm = confusion_matrix(y_test, pred, labels=clf.classes_)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
# disp.plot();





## Tensorflow

In [31]:
# data = df[['summary', 'gname']]
# ds = tf.data.Dataset.from_tensor_slices(dict(data))

# train, test = train_test_split(data, test_size=0.4)
# test, val = train_test_split(test, test_size=0.5)
# print(len(train), 'train examples')
# print(len(val), 'validation examples')
# print(len(test), 'test examples')
# print(len(data))

In [32]:
data = df[['summary', 'gname']]
# ds = tf.data.Dataset.from_tensor_slices(dict(data))

# train, test = train_test_split(data, test_size=0.4)
# test, val = train_test_split(test, test_size=0.5)
# print(len(train), 'train examples')
# print(len(val), 'validation examples')
# print(len(test), 'test examples')
# print(len(data))

# train_ds = tf.data.Dataset.from_tensor_slices((train['summary'].values, train['gname'].values))
# train_ds= tf.data.Dataset.from_tensor_slices(dict(train))
# val_ds= tf.data.Dataset.from_tensor_slices(dict(val))
# test_ds = tf.data.Dataset.from_tensor_slices(dict(test))
# train_ds = tf.data.Dataset.from_tensor_slices((train['summary'].values, train['gname'].values))
# print(train)

# max_features = 23502
# max_len  = 150
# embedding_dims = 100
# tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_features, lower=True)
# GLOVE_EMBEDDING =  "glove.6B.100d.txt"



In [33]:
#Data prep
# x_train = train['summary'].str.lower()
# y_train = train['gname'].values

In [34]:
# tokenizer.fit_on_texts(x_train)

# x_train =tokenizer.texts_to_sequences(x_train)
# x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train, maxlen=max_len)

In [35]:
# #create embedding matrix
# embeddings_index = {}

# with open(GLOVE_EMBEDDING, encoding='utf8') as f:
#     for line in f:
#         values = line.rstrip().rsplit(' ')
#         word = values[0]
#         embed = np.asarray(values[1:], dtype='float32')
#         embeddings_index[word] = embed

# word_index = tokenizer.word_index

# num_words = min(max_features, len(word_index) + 1)

# embedding_matrix = np.zeros((num_words, embedding_dims), dtype='float32')

# for word, i in word_index.items():

#     if i >= max_features:
#         continue

#     embedding_vector = embeddings_index.get(word)

#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

In [36]:
# input = tf.keras.layers.Input(shape=(max_len,))

# x = tf.keras.layers.Embedding(max_features, embedding_dims, weights=[embedding_matrix], trainable=False)(input)

In [37]:
# #Biderictional Layer

# x = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(32, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(x)

# x = tf.keras.layers.Conv1D(64, kernel_size=3, padding="valid", kernel_initializer="glorot_uniform")(x)

# avg_pool = tf.keras.layers.GlobalAveragePooling1D()(x)

# max_pool = tf.keras.layers.GlobalMaxPooling1D()(x)

# x = tf.keras.layers.concatenate([avg_pool, max_pool])

# preds = tf.keras.layers.Dense(6, activation="sigmoid")(x)

# x.summary()

# model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=1e-3), metrics=['accuracy'])


In [38]:
#convert the pandas object to a tensor
# train=tf.convert_to_tensor(train)
# type(data)
# train_data, test_data = tfds(train, split=["train", "test"], 
#                                   batch_size=-1, as_supervised=True)
# train_ds = tf.constant(([train['summary'], train['gname']]))
# x_train= tf.data.Dataset.from_tensor_slices(dict(x_train))
# train_examples, train_labels = tfds.as_numpy(x_train)

In [39]:
# for feat, targ in train.take(5):
#   print ('Features: {}, Target: {}'.format(feat, targ))

In [40]:
# embed = hub.load("https://tfhub.dev/google/nnlm-en-dim50/2")
# embeddings = embed(train_examples)

In [41]:
# print(embeddings.shape, embeddings.dtype)

In [42]:
# # build a model
# model = tf.keras.Sequential()
# model.add(hub_layer)
# model.add(tf.keras.layers.Dense(16, activation='relu'))
# model.add(tf.keras.layers.Dense(1))

# model.summary()

In [43]:
# # loss function and optimizer

# model.compile(optimizer = 'adam',
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [44]:
# mod_train = model.fit()(train_data.shuffle(10000).batch(512), 
#                         epochs =10
#                         validation_data = validation_data.batch(512),
#                         verbose=1)

In [45]:
# results = model.evaluate(test_data.batch(512), verbose=2)

# for name, value in zip(model.metric_names, results):
#     print("%s: %.3f" % (name, value))

## Multinomial Naive Bayes Classifier 

In [46]:

nbd_train = X_train['summary']
nbd_test = X_test['summary']
nby_train = y_train
nby_test = y_test


#vectorize summary data and transform to document term matrix
vectorizer = CountVectorizer(lowercase=True, stop_words='english')
vectorizer.fit(nbd_train)

nbd_train_dtm = vectorizer.transform(nbd_train)

In [47]:
#check out data
# vectorizer.get_feature_names()

In [48]:
# prep test data
nbd_test_dtm = vectorizer.transform(nbd_test)
nbd_test_dtm

<9875x27438 sparse matrix of type '<class 'numpy.int64'>'
	with 249002 stored elements in Compressed Sparse Row format>

In [49]:
#finally get to build the model
nb=MultinomialNB()


%time nb.fit(nbd_train_dtm, nby_train )

CPU times: user 95.3 ms, sys: 10 ms, total: 105 ms
Wall time: 106 ms


MultinomialNB()

In [50]:
# make prediction with test data
prediction = nb.predict(nbd_test_dtm)

In [51]:
metrics.accuracy_score(nby_test, prediction)

0.9475443037974683

In [52]:
# examine class distribution
print(y_test.value_counts())
# there is a majority class of 0 here, hence the classes are skewed

# calculate null accuracy (for multi-class classification problems)
# .head(1) assesses the value 1208
null_accuracy = y_test.value_counts().head(1) / len(y_test)
print('Null accuracy:', null_accuracy)

# Manual calculation of null accuracy by always predicting the majority class
print('Manual null accuracy:',(1208 / (1208 + 185)))

Taliban                                                        2030
Islamic State of Iraq and the Levant (ISIL)                    1470
Al-Shabaab                                                      904
Boko Haram                                                      681
Communist Party of India - Maoist (CPI-Maoist)                  507
New People's Army (NPA)                                         500
Maoists                                                         431
Revolutionary Armed Forces of Colombia (FARC)                   376
Tehrik-i-Taliban Pakistan (TTP)                                 366
Kurdistan Workers' Party (PKK)                                  346
Houthi extremists (Ansar Allah)                                 293
Al-Qaida in the Arabian Peninsula (AQAP)                        277
Muslim extremists                                               172
Liberation Tigers of Tamil Eelam (LTTE)                         170
Donetsk People's Republic                       

In [53]:
# print the confusion matrix
# metrics.confusion_matrix(y_test, y_pred_class)

In [54]:
# print message text for the false positives (ham incorrectly classified as spam)

X_test[y_pred_class > y_test]

# alternative less elegant but easier to understand
# X_test[(y_pred_class==1) & (y_test==0)]

NameError: name 'y_pred_class' is not defined

In [ ]:
# print message text for the false negatives (spam incorrectly classified as ham)

X_test[y_pred_class < y_test]
# alternative less elegant but easier to understand
# X_test[(y_pred_class=0) & (y_test=1)]


In [ ]:

# example false negative
X_test[3132]

In [120]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)

# Numpy Array with 2C
# left Column: probability class 0
# right C: probability class 1
# we only need the right column 
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

# Naive Bayes predicts very extreme probabilites, you should not take them at face value

NameError: name 'X_test_dtm' is not defined

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)
# AUC is useful as a single number summary of classifier performance
# Higher value = better classifier
# If you randomly chose one positive and one negative observation, AUC represents the likelihood that your classifier will assign a higher predicted probability to the positive observation
# AUC is useful even when there is high class imbalance (unlike classification accuracy)
# Fraud case
# Null accuracy almost 99%
# AUC is useful here

NLP

In [ ]:
# temp_corpus = X['summary'].dropna()
# corpus = temp_corpus.apply(lambda x: x.split(': ')[-1]).values
# # print(f'We have {len(corpus)} elements on the corpus\n\n')
# # print(f'Example 1: \n{corpus[1]}\n')
# # print(f'Example 2: \n{corpus[-1]}')

# corpus

In [ ]:
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.ensemble import RandomForestRegressor
# from scipy.stats import zscore
# from sklearn.cluster import DBSCAN
# from sklearn.covariance import EllipticEnvelope
# from sklearn.ensemble import IsolationForest
# from sklearn.neighbors import LocalOutlierFactor
# from sklearn.preprocessing import LabelEncoder
# from sklearn.svm import OneClassSVM
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import accuracy_score,f1_score,confusion_matrix
# import matplotlib.pyplot as plt
# from matplotlib.pyplot import pie
# %matplotlib inline
# import time
# import pandas_profiling
# import warnings
# warnings.filterwarnings("ignore")
# from sklearn.model_selection import train_test_split
# from sklearn.feature_selection import f_classif
# from xgboost import XGBClassifier
# from hyperopt import hp,Trials,fmin,tpe

In [23]:
df2 = data[data['gname'].isin(counts[counts > 700].index)]

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

df2['category_id'] = df2['gname'].factorize()[0]
category_id_df = df2[['gname', 'category_id']].drop_duplicates().sort_values('category_id')
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'gname']].values)
df2.head()
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.summary).toarray()
labels = df.gname
features.shape

<ipython-input-24-e6d41a60e823>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['category_id'] = df2['gname'].factorize()[0]


(32916, 21593)

In [25]:
from sklearn.feature_selection import chi2
import numpy as np
N = 2
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Product))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

# 'Al-Qaida in the Arabian Peninsula (AQAP)':
  . Most correlated unigrams:
. escorts
. escorting
  . Most correlated bigrams:
. essar steel
. zuweid town
# 'Al-Shabaab':
  . Most correlated unigrams:
. escorts
. escorting
  . Most correlated bigrams:
. essar steel
. zuweid town
# 'Basque Fatherland and Freedom (ETA)':
  . Most correlated unigrams:
. escorts
. escorting
  . Most correlated bigrams:
. essar steel
. zuweid town
# 'Boko Haram':
  . Most correlated unigrams:
. escorts
. escorting
  . Most correlated bigrams:
. essar steel
. zuweid town
# 'Communist Party of India - Maoist (CPI-Maoist)':
  . Most correlated unigrams:
. escorts
. escorting
  . Most correlated bigrams:
. essar steel
. zuweid town
# 'Houthi extremists (Ansar Allah)':
  . Most correlated unigrams:
. escorts
. escorting
  . Most correlated bigrams:
. essar steel
. zuweid town
# 'Irish Republican Army (IRA)':
  . Most correlated unigrams:
. escorts
. escorting
  . Most correlated bigrams:
. essar steel
. zuweid t